<a href="https://colab.research.google.com/github/Jollyhrothgar/deep_learning/blob/master/Deep_Learning_Lesson_1_set_up_your_environment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

To get this working on linux with a graphics card that supports CUDA accelleration, tensorflow, etc, we have to do many annoying steps, which I have attempted to document here.

## Step 0 - Start over after a system reboot resulted in everything going away.

After rebooting my system, attempting to restart the docker container created errors becuase the GPU was no longer recognized by docker as a valid device. This almost certainly means drives got updated and now everything is broken.

The docker stuff should all remain valid, assuming that we can get the GPU drivers fixed. Following [this guide here](https://collabnix.com/introducing-new-docker-cli-api-support-for-nvidia-gpus-under-docker-engine-19-03-0-beta-release/) for a CLI driven setup.

## Step 1 - Install the latest Nvidia Drivers from the "Proprietary Drives" Setting in Ubuntu

* Confirm its working by running

```bash
nvidia-smi
```

To get an output such as:

```
Mon Aug 31 16:41:35 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 435.21       Driver Version: 435.21       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:01:00.0  On |                  N/A |
|  0%   32C    P8    14W / 250W |    510MiB / 11176MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU       PID   Type   Process name                             Usage      |
|=============================================================================|
|    0      1359      G   /usr/lib/xorg/Xorg                            24MiB |
|    0      1496      G   /usr/bin/gnome-shell                          50MiB |
|    0      2406      G   /usr/lib/xorg/Xorg                           145MiB |
|    0      2544      G   /usr/bin/gnome-shell                         152MiB |
|    0      4018      G   ...AAAAAAAAAAAACAAAAAAAAAA= --shared-files   133MiB |
+-----------------------------------------------------------------------------+

```
## Step 2 - Set up docker, make sure your local user is added to the docker group.

## Step 3 - Get the NVIDIA docker container from tensorflow

[Tensorflow](https://www.tensorflow.org/install/docker) has documentation for accomplishing this, but the quick version is:

```bash
docker pull tensorflow/tensorflow:latest-gpu-jupyter
```

## Step 4 - Prepare / persist shit between runs so you can connect to colab


This gets run once:
```bash
docker run -ti --gpus all --rm -u $(id -u):$(id -g) -v "${HOME}/.config/jupyter:/.jupyter" tensorflow/tensorflow:latest-gpu-jupyter bash -c "source /etc/bash.bashrc && jupyter serverextension enable --py jupyter_http_over_ws"
```

## Step 5 - Run the docker container with the config:

```bash
docker run -it -u $(id -u):$(id -g) --gpus all -p 8888:8888 -v "${HOME}/.config/jupyter:/.jupyter" -v "${HOME}/workspace:/tf" tensorflow/tensorflow:latest-gpu-jupyter bash -c "source /etc/bash.bashrc && jupyter notebook --notebook-dir=/tf --ip 0.0.0.0 --no-browser --NotebookApp.allow_origin='https://colab.research.google.com'"
```

Obviously, you have to have your directory structure set up - for example, directorys like `${HOME}/workspace` and `${HOME}/.config/jupyter` have to exist.


## Step 6 Connect Local Runtime to Colab

Go to colab, and connect to a local runtime, pasting the string that looks like this:

`http://127.0.0.1:8888/?token=<long alphanumeric token>` into place for the backend URL, but replace `127.0.0.1` with `localhost`, since you tunneled the port earlier when you wrote `-p 8888:8888` in your docker run command.

## Step 7 - Make an executable to run your docker setup so you don't have to remember this shit.

## Step 8 - Run these commands with your local runtime connected.

You should see something that indicates your GPU is connected like:

`[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]`

## Step 9 - Write Down the Useful Links you Found

* https://jupyter-docker-stacks.readthedocs.io/en/latest/using/running.html
* https://docs.docker.com/engine/reference/run/
* https://stackoverflow.com/questions/61024722/how-to-use-google-colab-with-a-local-tensorflow-jupyter-server-using-powershell
* https://github.com/tensorflow/tensorflow/issues/25247#issuecomment-459644861
* https://stackoverflow.com/questions/58191215/how-to-add-python-libraries-to-docker-image
* https://research.google.com/colaboratory/local-runtimes.html

## Step 10: Vomit From Exhaustion

## Step 11: Setup Additional Dependencies

You might want to add stuff like pandas, matplotlib, seaborn, etc, to create a more full-featured analysis environment. You can do so by building a dockerfile based on another docker container.

You can have a file such as this:

```
# requirements.txt

pandas
sklearn
matplotlib
seaborn
```

And then build your own docker image based on a tensorflow image that works:

```
# Dockerfile

FROM tensorflow/tensorflow:latest-gpu-jupyter

RUN python -m pip install -r requirements.txt
```

In [ ]:
import tensorflow as tf

In [ ]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]